# 1. INTRODUÇÃO 
Você recebeu uma tarefa analítica de uma loja online internacional. Seu predecessor não conseguiu completá-la: ele lançou um teste A/B e depois desistiu (para começar uma fazenda de melancias no Tocantins). Ele deixou apenas as especificações técnicas e resultado dos testes.

**Descrição técnica**   
- Nome do teste: recommender_system_test  
- Grupos: A (controle) e B (funil de novos pagamentos)  
- Quando paparam de receber novos usuários: 21-12-2020  
- Data de término: 01-01-2021  
- Público: 15% de novos usuários da região da UE  
- Propósito do teste: testar mudanças relacionadas à introdução de um sistema de recomendação melhorado  
- Resultado esperado: em até 14 dias após o cadastro, usuários mostram uma conversão melhor nas visualizações de página do produto (o evento product_page event), em adicionar itens ao carrinho (product_cart) e de compras (purchase). Em cada etapa do funil product_page → product_cart → purchase, haverá ao menos 10% de aumento.  
- Número esperado de participantes do teste: 6000  

### Descrição dos dados

- final_ab_events_upd_us.csv — todos os eventos dos novos usuários dentro do período de 7 de dezembro de 2020 até 1 de janeiro de 2021
- final_ab_participants_upd_us.csv — tabela contendo os participantes do teste

**Estrutura**   
final_ab_events_upd_us.csv:  
&emsp;user_id  
&emsp;event_dt — data e hora do evento  
&emsp;event_name — nome da fonte do evento  
&emsp;details — dados adicionais sobre o evento (por exemplo, o total do pedido em USD para eventos purchase)  

**Estrutura**   
final_ab_participants_upd_us.csv:  
&emsp;user_id  
&emsp;ab_test — nome do teste  
&emsp;group — o grupo de teste ao qual o usuário pertencia

### Objetivo:
    O Objetivo é ver a taxa de conversão entre os grupos desdo cadastro até a compra a final do produto. Ver se realmente a partir das mudanças na loja online houve algum aumento na taxa de conversão entre grupo A e B,

# 2. DESENVOLVENDO O PROJETO 

Primeira parte é necessário importar as blibiotecas necessárias.

In [2]:
#Importando as bibliotecas
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

# 2.1 Explorando os daodos

O dataframe marketing_events não possui nenhum valor duplicado e nem ausente. Contudo, é preciso mudar o tipo do dados de star_dt e finish_dt para datetime

In [39]:
#importando o dataset final_ab_events_upd_us.csv
events = pd.read_csv('final_ab_events_upd_us.csv')

In [40]:
#lendo o dataframe events
display(events.head())
#verificando as informações do dataframe events
print(events.info())

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB
None


In [41]:
#verificando se há algum valor nulo no dataframe events
print(events.isnull().sum())
#verificando valores duplicados no dataframe events
print(events.duplicated().sum())

user_id            0
event_dt           0
event_name         0
details       363447
dtype: int64
0


Podemos ver que a coluna details possui valores ausentes, como esse valores são númericos e são utilizados com o fim de preencher dados adicionais sobre os eventos, vou preencher os valores nulos por 0 - que significa que nenhuma descrição adicional foi realizada. E também é necessário converter o tipo da coluna event_dt para datetime

In [42]:
#preenchendo o valor nulo da coluna details com 0
events['details'] = events['details'].fillna(0)
#verificando novamente valores nulos na coluna details
print(events['details'].isnull().sum())


#Convertendo a coluna event_dt para datetime
events['event_dt'] = pd.to_datetime(events['event_dt'])
#verificando o tipo da coluna event_dt
print(events.dtypes)

0
user_id               object
event_dt      datetime64[ns]
event_name            object
details              float64
dtype: object


In [43]:
#importnado o dataset final_ab_participants_upd_us.csv
participants = pd.read_csv('final_ab_participants_upd_us.csv')

In [44]:
#lendo o dataframe participants
display(participants.head())
#verificando as informações do dataframe participants
print(participants.info())

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB
None


In [45]:
#verificando se há algum valor nulo no dataframe participants
print(participants.isnull().sum())
#verificando valores duplicados no dataframe participants
print(participants.duplicated().sum())

user_id    0
group      0
ab_test    0
dtype: int64
0


O dataframe participants não precisa de nenhuma alteração 

### 2.1.1 Analisando os Funis 
Estudo da conversão em diferentes etapas do funil: analisar como funciona a conversão dos visitantes para compradores entre os dois grupos. Então primeiro será necessário juntar as tabelas final_ab_participants e final_ab_events_upd, por meio da função merge.

In [46]:
#unindo as tabelas events e participants, por meio da função merge do pandas
events_participants = pd.merge(events, participants, on='user_id', how='inner')

In [47]:
#filtrando o dataframe events_participants para considerar apenas o recommender_system_test
events_ab = events_participants[events_participants['ab_test'] == 'recommender_system_test']
print(events_ab.info())
display(events_ab.head())

<class 'pandas.core.frame.DataFrame'>
Index: 23909 entries, 1 to 102833
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     23909 non-null  object        
 1   event_dt    23909 non-null  datetime64[ns]
 2   event_name  23909 non-null  object        
 3   details     23909 non-null  float64       
 4   group       23909 non-null  object        
 5   ab_test     23909 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 1.3+ MB
None


,user_id,event_dt,event_name,details,group,ab_test
1,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99,A,recommender_system_test
6,3C5DD0288AC4FE23,2020-12-07 19:42:40,purchase,4.99,A,recommender_system_test
14,49EA242586C87836,2020-12-07 06:31:24,purchase,99.99,B,recommender_system_test
16,2B06EB547B7AAD08,2020-12-07 21:36:38,purchase,4.99,A,recommender_system_test
18,A640F31CAC7823A6,2020-12-07 18:48:26,purchase,4.99,B,recommender_system_test


Agora com nossos datasets unidos é preciso descobrir qual é percentual de usuários em cada grupo.

In [48]:
#vou agrupar os dados para termos a quantidade de usuários únicos por grupo de teste 
group_ab_users = events_ab.groupby('group')['user_id'].nunique().reset_index(name='qntd_usuarios')

# Criamos a coluna de porcentagem (Quantidade / Total)
group_ab_users['porcentagem'] = (group_ab_users['qntd_usuarios'] / group_ab_users['qntd_usuarios'].sum()) * 100

group_ab_users['rotulo_texto'] = group_ab_users['porcentagem'].apply(lambda x: f"{x:.2f}".replace('.', ',') + '%')

# Mostra os valores calculados no console
print(group_ab_users)

# 2. Plotar o gráfico com Plotly Express
fig = px.bar(
    group_ab_users, 
    x='group', 
    y='porcentagem',
    text='rotulo_texto', 
    color='group',
    title='Proporção de Usuários por Grupo (A vs B)',
    color_discrete_map={'A': '#1f77b4', 'B': '#ff7f0e'} )

# Estilização para ficar limpo

fig.update_traces(
    textposition='outside',
    textfont_size=12
)

fig.update_layout(
    yaxis_range=[0, 110],
    yaxis_ticksuffix="%",
    yaxis_title="Porcentagem de Usuários",
    template='plotly_white',
    showlegend=False # Opcional: remove legenda redundante já que o eixo X explica
)

fig.show()

  group  qntd_usuarios  porcentagem rotulo_texto
0     A           2747    74.748299       74,75%
1     B            928    25.251701       25,25%


Aqui há um erro gravíssimo na coleta dos dados, o ideal para um teste AB é uma distribuição 50/50 aqui temos uma divisão de 3:1, o que faz com que o grupo B probalisticamente possua maior margem para erros, qualquer comportamento atípico afetará a média. 

In [49]:
#Olhando para os eventos únicos no dataset events
print(events_ab['event_name'].value_counts())

event_name
login           10837
product_page     6702
purchase         3210
product_cart     3160
Name: count, dtype: int64


O caminho que o cliente faz é: login --> product_page --> product_cart --> purchase. Agora vamos descobrir o funil de uma etapa para a outra, descobrir a perda de clientes

In [50]:
#criando uma função para calcular o eventos dos dois grupos de usuários
def conversao_evento(df, group):
    # filtrando o grupo 
        # filtra o grupo
    df = df[df["group"] == group]

    # conta usuários únicos por evento
    eventos = (
        df.groupby("event_name")["user_id"]
        .nunique()
        .reindex(["login", "product_page", "product_cart", "purchase"], fill_value=0)
    )

    # evita divisão por zero
    def taxa(a, b):
        return round(a / b * 100, 2) if b > 0 else 0

    resultados = {
        "grupo": group,
        "n_login": eventos["login"],
        "n_page": eventos["product_page"],
        "n_cart": eventos["product_cart"],
        "n_purchase": eventos["purchase"],
        "conv_login_page": taxa(eventos["product_page"], eventos["login"]),
        "conv_page_cart": taxa(eventos["product_cart"], eventos["product_page"]),
        "conv_cart_purchase": taxa(eventos["purchase"], eventos["product_cart"]),
        "conv_total": taxa(eventos["purchase"], eventos["login"])
    }

    return resultados


In [51]:
#armazenando em uma variável os dados
conversao_a = conversao_evento(events_ab, 'A')
conversao_b = conversao_evento(events_ab, 'B')
#agora vamos criar os gráficos
    #primeiro teremos que transformar o dicionário em um dataframe
dados_conversao = []

for con in [conversao_a, conversao_b]:
    nome_grupo = f"Grupo {con['grupo']}"

    #Criei uma função lambda aqui dentro para formatar o texto com vírgula
    fmt = lambda x: f"{x:.2f}".replace('.', ',') + '%'
    
    dados_conversao.append({
        'Etapa': 'Login para Página do Produto',
        'Taxa de Conversão (%)':con['conv_login_page'],
        'Grupo': nome_grupo,
        'Texto': fmt(con['conv_login_page'])
    })
    dados_conversao.append({
        'Etapa': 'Página do Produto para Carrinho',
        'Taxa de Conversão (%)':con['conv_page_cart'],
        'Grupo': nome_grupo,
        'Texto': fmt(con['conv_page_cart'])
    })
    dados_conversao.append({
        'Etapa': 'Carrinho para Compra',
        'Taxa de Conversão (%)':con['conv_cart_purchase'],
        'Grupo': nome_grupo,
        'Texto': fmt(con['conv_cart_purchase'])
    })

df_plot = pd.DataFrame(dados_conversao)
fig = px.bar(
    df_plot,
    x='Etapa',
    y='Taxa de Conversão (%)',
    color='Grupo',
    barmode='group',
    title='Taxa de Conversão Sequencial (Etapa a Etapa)',
    text='Texto',
    color_discrete_map={'Grupo A': '#1f77b4', 'Grupo B': '#ff7f0e'}
)

# Estilização
fig.update_traces(
    textposition='outside', # Coloca o texto em cima da barra
    textfont_size=12
)

fig.update_layout(
    yaxis_tickformat='..0%', # Formato do eixo (ex: 50%)
    yaxis_title="Taxa de Conversão (%)",
    legend_title="Grupos",
    template='plotly_white',
    yaxis_range=[0, 110] 
)

fig.show()

#imprimindo os resultados da função conversao_evento para o grupo A
print(conversao_a)
#imprimindo os resultados da função conversao_evento para o grupo B
print(conversao_b)

{'grupo': 'A', 'n_login': 2747, 'n_page': 1780, 'n_cart': 824, 'n_purchase': 872, 'conv_login_page': 64.8, 'conv_page_cart': 46.29, 'conv_cart_purchase': 105.83, 'conv_total': 31.74}
{'grupo': 'B', 'n_login': 927, 'n_page': 523, 'n_cart': 255, 'n_purchase': 256, 'conv_login_page': 56.42, 'conv_page_cart': 48.76, 'conv_cart_purchase': 100.39, 'conv_total': 27.62}


Diferença entre os grupos:  
- O grupo A possui uma maior quantidade de usuários( A: 2.747; B: 927);
- O grupo A possui uma maior taxa de conversão de login -- product_page (A: 64,79%; B: 56,41%)
- O grupo B possui uma maior taxa de conversão de product_page -- product_cart (A: 46,29% e B: 48,75%)
- O grupo A possui uma maior taxa de conversão de product_cart -- purchase (A: 105,82% e B: 100,39%)
  
O último dado mostra que ou houve algum erro na coleta dos dados ou os clientes estão conseguindo pular a parte de colocar o produto no carrinho e indo diretamente para a compra. No entando, a conversão final é que no grupo A 31,74% dos usuários se mantém e no grupo B apenas 27,61% se mantém. Agora para vamos avaliar a distribuição de eventos ao decorrer do tempo.

In [52]:
#  Criando uma coluna apenas com o dia para trabalharmos no gráfico
events_ab['date'] = events_ab['event_dt'].dt.date

# Agrupar por data e grupo para contar os eventos
daily_events = events_ab.groupby(['date', 'group']).size().reset_index(name='event_count')

# 3. Criar gráfico interativo
fig = px.line(
daily_events,
x='date',
y='event_count',
color='group',
markers=True,
title='Distribuição do Número de Eventos por Dia (Grupos A e B)'
)

fig.update_layout(
        xaxis_title="Data",
        yaxis_title="Quantidade de Eventos",
        hovermode="x unified"
    )
fig.show()

C:\Users\muril\AppData\Local\Temp\ipykernel_32084\4217370032.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Podemos ver que o grupo A possui uma maior variação das compras, com o aumento continuo a partir do dia 14/12 e seu pico no dia 21/12. Podemos supor que devido a datas comemorativas as pessoas do grupo A se sentem motivadas a compar com a empresa, contudo no grupo não houve esse movimento. O próximo passo antes da analise do teste A/B e avaliar a distribuição entre as amostras

In [53]:
# Agrupa por usuário e grupo, conta as linhas de evento
events_per_user = events_ab.groupby(['user_id', 'group'])['event_name'].count().reset_index()
events_per_user.rename(columns={'event_name': 'event_count'}, inplace=True)

# Separar as amostras
sample_A = events_per_user[events_per_user['group'] == 'A']['event_count']
sample_B = events_per_user[events_per_user['group'] == 'B']['event_count']

# Teste de Levene
levene_stat, levene_p = stats.levene(sample_A, sample_B)

# Estatísticas descritivas para contexto
desc_A = sample_A.describe()
desc_B = sample_B.describe()

print(f"\nResultado do Teste de Levene:")
print(f"Estatística W = {levene_stat:.4f}")
print(f"p-valor = {levene_p:.4f}")

alpha = 0.05
if levene_p < alpha:
    print("\nConclusão: Rejeitamos a hipótese nula. As variâncias dos grupos são estatisticamente diferentes (heterocedasticidade).")
else:
    print("\nConclusão: Não rejeitamos a hipótese nula. As variâncias dos grupos são consideradas iguais (homocedasticidade).")


Resultado do Teste de Levene:
Estatística W = 4.1952
p-valor = 0.0406

Conclusão: Rejeitamos a hipótese nula. As variâncias dos grupos são estatisticamente diferentes (heterocedasticidade).


## 2.2 Teste A/B
Para evitar um falso negativo ou positivo irei realizar uma comparação multipla para ser possível entender qual seria a melhor porcetagem para nossa variavel alpha

In [54]:
#definindo variavel alpha padrão 
alpha = .05

# Realizando o teste t de Student
results = stats.ttest_ind(sample_A, sample_B, equal_var=False)  # Usando equal_var=False devido ao resultado do teste de Levene

bonferroni_alpha = alpha / 3

if (results.pvalue < bonferroni_alpha):
    print("Hipótese nula rejeitada para os grupos А e B")
else:
    print("Hipótese nula não rejeitada para os grupos А e B")

Hipótese nula rejeitada para os grupos А e B


Mesmo utilizando a correção de Bonferroni ainda podemos ver que possuimos diferenças entre a distribuição das amostragens. Para entender melhor onde estão as diferenças entre elas eu vou calcular a distribuição levando em conta cada etapa do funil de compra. Esse olhar mais cirurgico irá permiter entender onde estão as fudamentais mudanças entre os grupos e avaliar se realmente houve um aumento na captação e retenção do grupo B em relação ao A

In [55]:
def testar_hipotese_ab(df, etapa_funil, alpha=0.016):
    
    print(f"--- Testando Diferença para: {etapa_funil} ---")
    # Total de usuários em cada grupo
    n_users_A = df[df['group'] == 'A']['user_id'].nunique()
    n_users_B = df[df['group'] == 'B']['user_id'].nunique()
    
    # Usuários que converteram na etapa específica
    convertidos_A = df[(df['group'] == 'A') & (df['event_name'] == etapa_funil)]['user_id'].nunique()
    convertidos_B = df[(df['group'] == 'B') & (df['event_name'] == etapa_funil)]['user_id'].nunique()
    
    # Test z de proporções
    count = np.array([convertidos_A, convertidos_B])
    nobs = np.array([n_users_A, n_users_B])
    
    z_stat, p_value_z = proportions_ztest(count, nobs)
    
    print(f"Grupo A: {convertidos_A}/{n_users_A} ({convertidos_A/n_users_A:.2%})")
    print(f"Grupo B: {convertidos_B}/{n_users_B} ({convertidos_B/n_users_B:.2%})")
    print(f"Z-test p-valor: {p_value_z:.4f}")
    
    if p_value_z < alpha:
        print("Rejeitamos H0: Há diferença significativa na conversão.")
    else:
        print("Não rejeitamos H0: Não há diferença significativa.")
        
    print("-" * 30)

def testar_media_eventos(df, alpha=0.016):
    """
    Realiza o Teste T de Welch para comparar a média de eventos por usuário.
    """
    print("--- Testando Diferença na Média de Eventos (Welch's t-test) ---")
    
    # Contar eventos por usuário
    eventos_por_user = df.groupby(['user_id', 'group']).size().reset_index(name='contagem')
    
    amostra_A = eventos_por_user[eventos_por_user['group'] == 'A']['contagem']
    amostra_B = eventos_por_user[eventos_por_user['group'] == 'B']['contagem']
    
    # --- TESTE T DE WELCH (equal_var=False) ---
    # Fundamental porque o Levene indicou variâncias diferentes
    t_stat, p_value_t = stats.ttest_ind(amostra_A, amostra_B, equal_var=False)
    
    print(f"Média A: {amostra_A.mean():.2f} | Média B: {amostra_B.mean():.2f}")
    print(f"Welch t-test p-valor: {p_value_t:.4f}")
    
    if p_value_t < alpha:
        print("Rejeitamos H0: As médias são estatisticamente diferentes.")
    else:
        print("Não rejeitamos H0: As médias são iguais.")

In [56]:
testar_hipotese_ab(events_ab, 'purchase')
testar_media_eventos(events_ab)


--- Testando Diferença para: purchase ---
Grupo A: 872/2747 (31.74%)
Grupo B: 256/928 (27.59%)
Z-test p-valor: 0.0176
Não rejeitamos H0: Não há diferença significativa.
------------------------------
--- Testando Diferença na Média de Eventos (Welch's t-test) ---
Média A: 6.78 | Média B: 5.69
Welch t-test p-valor: 0.0000
Rejeitamos H0: As médias são estatisticamente diferentes.


In [57]:
testar_hipotese_ab(events_ab, 'product_cart')

--- Testando Diferença para: product_cart ---
Grupo A: 824/2747 (30.00%)
Grupo B: 255/928 (27.48%)
Z-test p-valor: 0.1453
Não rejeitamos H0: Não há diferença significativa.
------------------------------


In [58]:
testar_hipotese_ab(events_ab, 'product_page')

--- Testando Diferença para: product_page ---
Grupo A: 1780/2747 (64.80%)
Grupo B: 523/928 (56.36%)
Z-test p-valor: 0.0000
Rejeitamos H0: Há diferença significativa na conversão.
------------------------------


# CONCLUSÃO
A conclusão que eu chego é: **o teste falhou**  

Desde de sua coleta, temos um total de clientes no grupo A  (75%) 3 vezes maior que o grupo B (25%), o que resulta um dado de amostragem que não corresponde um ao outro, como o teste de leneve demonstrou. O que resultada nas seguintes discrepância entre os dados

### Na analise detalhada
**Visualização das página (product_page)**
Resultado: queda de 64,80% (A) para 56,36%(B)
P-Valor: 0,0000
Conclusão: O novo sistema de recomendação não é eficiente em fazer os usuários se interessarem pelo produto.

**Etapa do carrinho (product_cart)**
Resultado: queda de 30,00% (A) para 27,48%(B)
P-Valor: 0,1453
Conclusão: Não houve reijação da hipotese nula. Porém como o objetivo era o aumento dos resultados também podemos observar uma falha.

**Compra (purchase)**
Resultado: queda de 31,74% (A) para 27,59%(B)
P-Valor: 0,0176
Conclusão: O sistema B converte menos que o sistema A.


Ou seja, o estudo mostra que a melhor opção seria manter o sistema A por hora. Porque para comprovar que o sistema B é mais eficiente ou não precisa ser feito outra coleta de dados, de forma adequada para não termos esses tipos de resultados e podemos comparar os dois grupos